In [1]:
import Reader3 as Reader
from astropy.table import Table, Column
import numpy as np
import os
from astropy.io import fits

Using uproot:  5.3.10


# Add pulsar phase to DL3 FITS files

by matching 'EVENT ID' with fully analyzed ROOT files containing phase information

In [2]:
fits_dir = '.' # Path to the directory containing DL3 FITS files
root_dir = '../data/all_root_data/' # Path to the directory containing melibea ROOT files with phase information

fits_files = [filename for filename in os.listdir(fits_dir) if filename.endswith('.fits')]
root_files = [filename for filename in os.listdir(root_dir) if filename.endswith('.root')]

In [3]:
def addphase(root_path, fits_path):
    root = Reader.Reader(root_path,{'daqnum':'MRawEvtHeader_2.fStereoEvtNumber','phase':'MPhase2_1.fPhase'})

    with fits.open(fits_path, mode='update') as hdulist:
        hdu_events = hdulist[1]
        fits_table = Table(hdu_events.data)

        if 'PHASE' in fits_table.colnames:
            print(f"Skipping fits file: {fits_path}. 'PHASE' column already exists.")
            hdulist.close()
            return
        
        id_to_val = {row['daqnum']: row['phase'] for row in root}

        phase_fits = np.full(len(fits_table), np.nan)

        for i, id in enumerate(fits_table['EVENT_ID']):
            if id in root['daqnum']:
                phase_fits[i] = id_to_val[id]

        new_col = fits.Column(name='PHASE', format='D', unit='', array=phase_fits)
        orig_cols = hdulist[1].data.columns
        combined = fits.BinTableHDU.from_columns(orig_cols + new_col)

        hdulist[1].data = combined.data
        hdulist.flush()

In [ ]:
for fits_file in fits_files:
    if fits_file.endswith('.fits'):
        run = fits_file.split('_')[1]
        fits_path = os.path.join(fits_dir, fits_file)
        
        print(f"Processing fits file: {fits_path}")

        # search for corresponding root files
        found_root = False
        for root_file in root_files:
            root_run = root_file.split('_')[1].split('.')[0]
        
            if root_run == run:
                root_path = os.path.join(root_dir, root_file)
                found_root = True
                break
    
        if found_root:
            try:
                print(f"Corresponding root file: {root_path}")
                
                addphase(root_path, fits_path)
                
                print("Processing complete.", '\n')
                
            except Exception as e:
                print(f"Error processing files {fits_path}, {root_file}: {e}")

        else:
            print(f"No corresponding root file found for fits file: {fits_file}", '\n')